### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn4 import NN4
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.sync_sgd import sync_sgd
# TODO: Import all the necessary code from mlcvlab package as you need... 

### 3. Set Seed

In [3]:
np.random.seed(42)

### 4. Helper functions

In [4]:
def load_dataset():
    x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    y = y.astype(np.int64)
    return x,y

def prepare_data(x, y):
    y = y & 1
    return x, y

def split_train_test(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=10000, random_state=25)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    return X_train, X_test, y_train, y_test

def minibatch(X_train,y_train,K):
    batches = X_train.shape[0] / K
    X_train_batches = np.array_split(X_train, batches)
    y_train_batches = np.array_split(y_train, batches)
    return X_train_batches, y_train_batches

def initialize_model():
    W0 = np.random.randn(785, 500) * np.sqrt(2/785)
    W1 = np.random.randn(500, 100) * np.sqrt(2/500)
    W2 = np.random.randn(100, 50) * np.sqrt(2/100)
    W3 = np.random.randn(50, 1) * np.sqrt(2/50)
    print(f"Size of W0 : {W0.shape}, Size of W1 : {W1.shape}, Size of W2 : {W2.shape}, Size of W3 : {W3.shape}")
    four_layer_nn  = NN4(True, 0.75)
    four_layer_nn.layers[0].W = W0
    four_layer_nn.layers[1].W = W1
    four_layer_nn.layers[2].W = W2
    four_layer_nn.layers[3].W = W3

    return four_layer_nn

def train_model(model, X_train_batches, y_train_batches):
    return sync_sgd(model, X_train_batches, y_train_batches, lr=0.01, mode='train')

def test_model(model, X_test, y_test):
    accuracy = None
    total = 0
    for i in range(X_test.shape[0]):
        X_sample_test = X_test[i, :]
        X_sample_test = np.append(X_sample_test, -1)
        y_actual = y_test[i]
        y_hat = model.nn4(X_sample_test, mode='test')
        y_hat = (y_hat >= 0.5) * 1
        if y_hat == y_actual:
            total = total + 1

    accuracy = total/len(X_test)
    
    return accuracy

### 5. Run the program

In [5]:
#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)

#initialize model
model = initialize_model()

K = 100
X_train_batches, y_train_batches = minibatch(X_train,y_train,K)

model_async = train_model(model, X_train_batches, y_train_batches)  

accuracy = test_model(model_async, X_test, y_test)
print(f"Completed testing model - Accuracy : {accuracy}")

Size of W0 : (785, 500), Size of W1 : (500, 100), Size of W2 : (100, 50), Size of W3 : (50, 1)
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
Completed testing model - Accuracy : 0.5796
